In [1]:
import boto3
import matplotlib.pyplot as plt
import re
import os
import glob
import scipy.io
import numpy as np
import io
import numpy as np
import pandas as pd
from PIL import Image
from multiprocessing import Pool

# Constants

In [2]:
bucket_name  = 'edp.engie-digital.prod.instance1.prod.data'
num_partitions = 53
num_cores = 28 

# point de la station

In [3]:
geocord = scipy.io.loadmat('./geographicCoordinate.mat')['geographicCoordinate']
lat = geocord['latitude'][0][0]
lon = geocord['longitude'][0][0]
latreshape = lat[35:575, 35:795]
lonreshape = lon[35:575, 35:795]


In [4]:
latitude_walon = 42.7983
longitude_walon = 2.8856
distance = np.sqrt((latreshape - latitude_walon)**2+(lonreshape-longitude_walon)**2)
pixel_block_matching = np.where(distance == distance.min())
pix_block_matching = pixel_block_matching[0][0], pixel_block_matching[1][0]

In [5]:
distance = np.sqrt((lat - latitude_walon)**2+(lon-longitude_walon)**2)
pixel = np.where(distance == distance.min())
pix = pixel[0][0], pixel[1][0]

In [6]:
pix_block_matching

(492, 376)

In [7]:
pix

(527, 411)

# Images

In [8]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucket_name)
s3_client = boto3.client('s3')

In [9]:
%%time
processed_images = [x.key for x  in bucket.objects.filter(Prefix='LCV_SATELLITE_IMAGE_24/infrared_images/processed/')]

CPU times: user 3.54 s, sys: 95.5 ms, total: 3.64 s
Wall time: 7.46 s


In [10]:
processed_images = np.sort(processed_images)

In [11]:
available_images = {str(pd.to_datetime(image[-19:-4])):image for image in processed_images}

In [12]:
def parallelize_dataframe(list_archives, func):
    df_split = np.array_split(list_archives, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [13]:
def compute_irradiance_feature(processed_images):
    time = []
    outVx = []
    outVy = []
    out_irradiance = []
    zenith = []
    azimuth = []

    for image in processed_images:
        current = str(pd.to_datetime(image[-19:-4])+pd.Timedelta(minutes=15))
        if current in available_images.keys():
            img_1 = s3_client.get_object(Bucket=bucket_name, Key=image)
            img_1_refined = np.array(Image.open(img_1['Body']).convert("L"))
            img_2 = s3_client.get_object(Bucket=bucket_name, Key=available_images[current])
            img_2_refined = np.array(Image.open(img_2['Body']).convert("L"))

            Vx, Vy, X, Y = block_match(img_1_refined,img_2_refined)
            Vx_smoothed, Vy_smoothed = smoothing_vectors(Vx, Vy)
            Vx_interp, Vy_interp, X_interp, Y_interp = interpolate_vectors(Vx_smoothed, Vy_smoothed, X, Y)
            outVx.append(Vx_interp[pix_block_matching[0]-5: pix_block_matching[0]+6,pix_block_matching[1]-5: pix_block_matching[1]+6])
            outVy.append(Vy_interp[pix_block_matching[0]-5: pix_block_matching[0]+6,pix_block_matching[1]-5: pix_block_matching[1]+6])
            out_irradiance.append(img_2_refined[pix[0]-5: pix[0]+6,pix[1]-5: pix[1]+6])
            time.append(current)
            
            z,a = solar_angles_computation(np.datetime64(current),
                                               lat[pix[0]-5: pix[0]+6,pix[1]-5: pix[1]+6],
                                               lon[pix[0]-5: pix[0]+6,pix[1]-5: pix[1]+6])
            zenith.append(z)
            azimuth.append(a)
        else:
            continue
    return pd.DataFrame({'time':time, 'block_matching_vx': outVx, 
                         'block_matching_vy': outVy, 'brightness':out_irradiance,
                         'zenith':zenith, 'azimuth': azimuth
                        }) 

In [ ]:
%%time
computed_irradiance_features = parallelize_dataframe(processed_images, compute_irradiance_feature)

In [22]:
computed_irradiance_features.shape

(21493, 6)

In [23]:
!ls

adnot_data.ipynb
adnot_data_train.csv
block_matching_brigtness_temperature_solar_angles_data.ipynb
block_matching_compute.ipynb
clearsky_ref.ipynb
computed_cloud_index_walon.csv
geographicCoordinate.mat
rename_infrared.ipynb


In [25]:
computed_irradiance_features.to_csv('computed_irradiance_features.csv', index = False)

In [27]:
cloud_index = pd.read_csv('computed_cloud_index_walon.csv')

In [32]:
cloud_index.date = pd.to_datetime(cloud_index.date)

In [50]:
computed_irradiance_features.columns = ['azimuth', 'block_matching_vx', 'block_matching_vy', 'brightness',
       'date', 'zenith']

In [52]:
computed_irradiance_features.date = pd.to_datetime(computed_irradiance_features.date)

In [53]:
df = pd.merge(computed_irradiance_features, cloud_index, on='date')

In [59]:
df.to_csv('dataframe_input_random_forest.csv', index = False)

In [2]:
data = pd.read_csv('dataframe_input_random_forest.csv')

In [3]:
data.head()

,azimuth,block_matching_vx,block_matching_vy,brightness,date,zenith,cloud_index_walon
0,[[109.61015815 109.60888455 109.60760772 109.6...,[[-0.84162397 -0.85872785 -0.87583172 -0.89293...,[[0.49077446 0.4995504 0.50832635 0.51710229 ...,[[106 95 99 100 98 96 97 101 100 97 102]...,2017-01-22 15:30:00,[[93.27648497 93.28440141 93.29231819 93.30023...,[[0.07602339 0.10588235 0.13173653 0.11643836 ...
1,[[109.53907817 109.53642355 109.5337655 109.5...,[[-0.81299615 -0.82750894 -0.84202174 -0.85653...,[[0.51081341 0.52218043 0.53354746 0.54491448 ...,[[101 93 97 94 93 98 104 105 100 94 109]...,2017-01-22 15:45:00,[[96.80580811 96.8136967 96.82158511 96.82947...,[[0. 0. 0. 0.03797468 ...
2,[[109.54614777 109.54209206 109.53803266 109.5...,[[-0.85963162 -0.86633112 -0.87303063 -0.87973...,[[0.23691368 0.2412274 0.24554112 0.24985483 ...,[[102 100 103 106 110 109 105 102 101 102 114]...,2017-01-22 16:00:00,[[100.33588665 100.34377771 100.35166805 100.3...,[[0.53757225 0.51829268 0.50967742 0.45859873 ...
3,[[109.63362497 109.62813952 109.62265003 109.6...,[[-0.73276688 -0.7456406 -0.75851431 -0.77138...,[[0.20544951 0.20664756 0.2078456 0.20904364 ...,[[106 105 107 105 104 104 106 108 108 107 105]...,2017-01-22 16:15:00,[[103.86498849 103.87291321 103.88083665 103.8...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0...
4,[[109.80539479 109.79844175 109.79148428 109.7...,[[-0.71167976 -0.73012137 -0.74856297 -0.76700...,[[0.33185455 0.3320181 0.33218165 0.3323452 ...,[[112 100 107 106 104 105 107 108 108 108 112]...,2017-01-22 16:30:00,[[107.39130245 107.39929346 107.40728262 107.4...,[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]\n [0. 0. 0...


In [61]:
len(set.intersection(set(computed_irradiance_features.date), set(cloud_index.date)))

15617

# Utils

In [16]:
import numpy as np
from scipy.interpolate import griddata
from scipy import signal
import json


# Parameters & config
config = {
  "bucket_S3_name" : "octo-rivesaltes",
  "image_sat_visual_complete" : "https://en.sat24.com/image?type=visual5Complete&type=visual5Complete&region=fr&region=fr&timestamp=",
  "image_sat_infra_polair" : "https://en.sat24.com/image?type=infraPolair&type=infraPolair&region=fr&region=fr&timestamp=",
  "block_matching_size_target_area" : 41,
  "block_matching_patch" : 15,
  "block_matching_X" : 615,
  "block_matching_Y" : 845
}


size_target_area = config['block_matching_size_target_area']
p = config['block_matching_patch']
img_size = (config['block_matching_X'], config['block_matching_Y'])

q = int(np.floor(size_target_area/2))

x = np.arange(p+q, img_size[0]-(p+q)+1, q)
y = np.arange(p+q, img_size[1]-(p+q)+1, q)

xvgrid, yvgrid = np.meshgrid(x, y)


def _power2(x):
    return np.power(x, 2)


def _create_gaussian_kernel(sigma, rayon):
    sz = 2 * rayon + 1
    gaussianKernelFactor = 0
    kernel = np.zeros((sz, sz))

    for ky in range(-rayon, rayon):
        for kx in range(-rayon, rayon):
            e = np.exp(- (kx * kx + ky * ky) / (2 * sigma * sigma))
            gaussianKernelFactor = gaussianKernelFactor + e
            kernel[kx + rayon + 1, ky + rayon + 1] = e
    for ky in range(-rayon, rayon):
        for kx in range(-rayon, rayon):
            kernel[kx + rayon + 1, ky + rayon + 1] = kernel[kx + rayon + 1, ky + rayon + 1] / gaussianKernelFactor

    return kernel


def patchify(img, patch_shape):
    img = np.ascontiguousarray(img)  # won't make a copy if not needed
    X, Y = img.shape
    x, y = patch_shape
    shape = ((X-x+1), (Y-y+1), x, y) # number of patches, patch_shape
    # The right strides can be thought by:
    # 1) Thinking of `img` as a chunk of memory in C order
    # 2) Asking how many items through that chunk of memory are needed when indices
    #    i,j,k,l are incremented by one
    strides = img.itemsize*np.array([Y, 1, Y, 1])
    return np.lib.stride_tricks.as_strided(img, shape=shape, strides=strides)


def block_match(img_1, img_2):
    """
    Apply block matching on two images
    :param numpy.array img_1: image with grey levels
    :param numpy.array img_2: binary encoding of satellite image
    :return: numpy.array X,Y,Vx,Vx: Vectors of images block_matched
    """

    if((img_1.shape != img_size)  | (img_2.shape != img_size)):
        raise IndexError('Wrong image size')

    vectorized_power = np.vectorize(_power2)
    img_1_refined = vectorized_power(max(img_1.max(), img_2.max()) - img_1)
    img_2_refined = vectorized_power(max(img_1.max(), img_2.max()) - img_2)

    X = xvgrid.flatten()
    Y = yvgrid.flatten()
    Vx = np.array([])
    Vy = np.array([])

    for x_point, y_point in zip(X, Y):
        target_area = img_2_refined[x_point - q:x_point + q + 1, y_point - q:y_point + q + 1]
        img_to_patch = img_1_refined[x_point - (q + p):x_point + (q + p + 1), y_point - (q + p):y_point + (q + p + 1)]
        results = patchify(img_to_patch, target_area.shape)
        error = abs(results - target_area)
        costs = np.apply_over_axes(np.mean, error, [2, 3]).reshape(error.shape[0:2])
        x_patch_most_likely, y_patch_most_likely = np.where(costs == costs.min())
        Vx = np.append(Vx, p - x_patch_most_likely[0])
        Vy = np.append(Vy, p - y_patch_most_likely[0])

    return Vx, Vy, X, Y


def smoothing_vectors(Vx, Vy):
    """
    smooth values from vectors Vx and Vy
    :param numpy.ndarray Vx: Vectors of coordinates on axis X
    :param numpy.ndarray Vy: Vectors of coordinates on axis Y
    :param tuple shape : Size to reshape the vectors in a matrix (cf xvgrid)
    :return: numpy.ndarray Vx,Vx: Vectors of coordinates smoothed
    """

    kernel = _create_gaussian_kernel(sigma=1, rayon=2)
    Vx = signal.convolve2d(Vx.reshape(xvgrid.shape), kernel, 'same')
    Vy = signal.convolve2d(Vy.reshape(xvgrid.shape), kernel, 'same')

    return Vx, Vy


def interpolate_vectors(Vx, Vy, X, Y):
    """
    intepolate values from vectors Vx and Vy
    :param numpy.ndarray X: points of coordinates on axis X
    :param numpy.ndarray Y: points of coordinates on axis Y
    :param numpy.ndarray Vx: Vectors of coordinates on axis X
    :param numpy.ndarray Vy: Vectors of coordinates on axis Y
    :param tuple shape : Size to reshape the vectors in a matrix (cf xvgrid)
    :return: numpy.ndarray Vx,Vx: Vectors of coordinates interpolated
    """

    grid_x, grid_y = np.mgrid[min(X):max(X), min(Y):max(Y)]
    arr = np.array((X.reshape(xvgrid.shape)[~np.isnan(Vx)].flatten(),
                    Y.reshape(xvgrid.shape)[~np.isnan(Vy)].flatten())).transpose()

    Vx = griddata(arr, Vx[~np.isnan(Vx)].flatten(), (grid_x, grid_y), method='linear')
    Vy = griddata(arr, Vy[~np.isnan(Vy)].flatten(), (grid_x, grid_y), method='linear')

    return Vx, Vy, grid_x, grid_y


In [17]:
import numpy as np
import pandas as pd


def day_of_the_year_computation(universal_time):
    """
    Computes the floating value for the day of the year
    :param numpy.datetime64 universal_time:
    :return: float day of year:
    """
    time_object = pd.to_datetime(universal_time)
    return time_object.dayofyear + time_object.hour/24.0 + time_object.minute/(24.0*60)


def solar_angles_computation(universal_time, longitude, latitude):
    """
    Computes the zenith matrix associated with longitude, latitude at a given universal time,
    reference NOAA Global Monitoring Division
    :param numpy.datetime64 universal_time: universal time
    :param numpy.ndarray longitude: array of the considered longitude
    :param numpy.ndarray latitude: array of the considered latitude
    :return:numpy.ndarray zenith: matrix of the zenith in degrees
    :return:numpy.ndarray azimuth: matrix of the azimuth in degrees
    """
    if longitude.shape == latitude.shape:
        day_of_year = day_of_the_year_computation(universal_time)
        x = 2 * np.pi / 365 * (day_of_year - 1)  # day of year un radian, named x because heavily referenced

        eqtime = (0.000075 + 0.001868 * np.cos(x) - 0.032077 * np.sin(x) - 0.014615 * np.cos(2 * x)
                  - 0.040849 * np.sin(2 * x)) * ((24 * 60) / (2 * np.pi))

        solar_declination = 0.006918 - 0.399912 * np.cos(x) + 0.070257 * np.sin(x) \
                            - 0.006758 * np.cos(2 * x) + 0.000907 * np.sin(2 * x) \
                            - 0.002697 * np.cos(3 * x) + 0.001480 * np.sin(3 * x)

        longitude_corrected = 4*longitude
        offset = (longitude_corrected + eqtime).astype('timedelta64[m]')
        true_solar_time = offset + universal_time
        true_solar_time = [x.astype(object) for x in true_solar_time.flatten()]
        true_solar_time_minutes = [(lambda time: (time.hour*60 + time.minute + time.second/60))(time)
                                   for time in true_solar_time]

        hour_angle = np.array([(x/4 - 180)*np.pi/180 for x in true_solar_time_minutes]).reshape(latitude.shape)
        zenith = np.arccos(np.cos(solar_declination)*np.cos(latitude*np.pi/180)*np.cos(hour_angle)
                           + np.sin(solar_declination)*np.sin(latitude*np.pi/180))

        azimuth = np.arccos((np.sin(solar_declination) * np.cos(latitude * np.pi / 180) - np.cos(solar_declination)
                           * np.sin(latitude * np.pi / 180) * np.cos(hour_angle)) / np.sin(zenith))

        return zenith * 180 / np.pi, azimuth * 180 / np.pi
    else:
        return ValueError


